In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime
import time

# 🚨 เพิ่ม importlib สำหรับการ reload
import importlib

# ==========================================================
# 1️⃣ Import Libraries และ Logic Files
# ==========================================================
import detect_wyckoff
import detect_flags

# 🚨 บังคับให้โหลดไฟล์ Logic ใหม่ (ป้องกันการใช้โค้ดเก่า)
importlib.reload(detect_wyckoff)
importlib.reload(detect_flags)

# ดึงฟังก์ชันมาจากไฟล์ที่เพิ่งถูก reload
from detect_wyckoff import detect_wyckoff_events
from detect_flags import detect_flag_patterns

# ==========================================================
# ฟังก์ชันดึงข้อมูลจาก Cassandra
# ==========================================================
def get_candlestick_data(symbol: str, days: int = 30) -> pd.DataFrame:
    """ดึงข้อมูลราคาและ Volume จาก Cassandra และเปลี่ยนชื่อคอลัมน์ให้เป็นชื่อสั้น"""
    
    # ⚠️ ตรวจสอบว่า Cassandra Server ได้รันอยู่ (127.0.0.1:9042)
    # (โค้ดเชื่อมต่อ Cassandra เหมือนเดิม...)
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect('data_stock')
    
    end_time = datetime.now()
    start_time = end_time - pd.Timedelta(days=days)
    
    # ใช้ ALIAS ใน CQL query เพื่อแปลงชื่อคอลัมน์
    query = f"""
    SELECT time, open_price AS open, high_price AS high, low_price AS low, close_price AS close, volume 
    FROM candlestick_data 
    WHERE symbol = '{symbol}' AND time >= %s ALLOW FILTERING;
    """
    
    rows = session.execute(query, [start_time])
    
    df = pd.DataFrame(list(rows))
    if df.empty:
        raise ValueError(f"ไม่พบข้อมูลสำหรับสัญลักษณ์ '{symbol}' ในช่วง {days} วัน")
        
    df.set_index('time', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    return df.sort_index()

In [2]:
# ==========================================================
# ฟังก์ชันวาดกราฟ Plotly Candlestick (พื้นฐาน)
# ==========================================================
def create_candlestick_fig(df: pd.DataFrame, symbol: str, title_suffix: str = "") -> go.Figure:
    """สร้าง Figure พื้นฐานสำหรับ Candlestick และ Volume"""
    fig = go.Figure()
    
    # 🔹 แท่งเทียนราคา
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Price'
    ))
    
    # 🔹 Volume bar
    fig.add_trace(go.Bar(
        x=df.index,
        y=df['volume'],
        marker_color='rgba(128,128,128,0.3)',
        yaxis='y2', 
        name='Volume'
    ))
    
    # 🔹 ปรับ Layout
    fig.update_layout(
        title=f'📈 {symbol} - Candlestick Chart {title_suffix}',
        xaxis_rangeslider_visible=False,
        xaxis=dict(type='date'),
        yaxis=dict(title='Price'),
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right',
            showgrid=False,
            domain=[0, 0.2] 
        ),
        yaxis1=dict(domain=[0.2, 1]),
        height=700,
        template='plotly_white'
    )
    return fig

# ==========================================================
# ฟังก์ชันวาด Wyckoff Cycle บน Plotly (ใช้ go.Figure)
# ==========================================================
def plot_wyckoff_cycle(df: pd.DataFrame, events: dict, symbol: str):
    fig = create_candlestick_fig(df, symbol, title_suffix="with Wyckoff Cycle")

    # ------------------------------------------------------------------
    # 1. วาด Wyckoff Cycle 4 ระยะ (Accumulation, Markup, Distribution, Markdown)
    # ------------------------------------------------------------------
    for pattern in events['WYCKOFF_CYCLE']:
        start_time = df.index[pattern['start_idx']]
        end_time = df.index[pattern['end_idx']]
        
        if pattern['phase'] == "ACCUMULATION" or pattern['phase'] == "MARKUP":
            color = 'green'
        else: # DISTRIBUTION, MARKDOWN
            color = 'red'

        # วาดแถบพื้นหลัง (Vertical Span)
        fig.add_vrect(
            x0=start_time, x1=end_time,
            fillcolor=color, 
            opacity=0.1 if 'TR' in pattern['phase'] else 0.25, # TR Zone จะทึบน้อยกว่า
            layer="below", line_width=0
        )
        
        # กำกับข้อความ
        y_pos = df['high'].max() if pattern['phase'] in ["ACCUMULATION", "MARKUP"] else df['low'].min()
        y_shift = 10 if pattern['phase'] in ["ACCUMULATION", "MARKUP"] else -10
        
        fig.add_annotation(
            x=start_time, y=y_pos, text=pattern['phase'],
            showarrow=False,
            font=dict(color='dark' + color, size=10, weight='bold'),
            yshift=y_shift
        )

    # ------------------------------------------------------------------
    # 2. วาดจุดเหตุการณ์สำคัญ (SC, BC, Spring, UTAD)
    # ------------------------------------------------------------------
    
    def add_event_markers(indices, name, color, is_low_point):
        if not indices: return
        event_df = df.iloc[indices]
        y_data = event_df['low'] if is_low_point else event_df['high']
        
        fig.add_trace(go.Scatter(
            x=event_df.index, y=y_data,
            mode='markers+text',
            marker=dict(symbol='circle', size=10, color=color, line=dict(width=2, color=color)),
            text=[name] * len(indices),
            textposition='top center' if not is_low_point else 'bottom center',
            textfont=dict(color=color, size=12),
            name=name
        ))

    add_event_markers(events['SC'], 'SC', 'green', True)
    add_event_markers(events['BC'], 'BC', 'red', False)
    add_event_markers(events['Spring'], 'Spring', 'purple', True)
    add_event_markers(events['UTAD'], 'UTAD', 'orange', False)

    fig.show()

# ==========================================================
# ฟังก์ชันวาด Flag Patterns บน Plotly
# ==========================================================
def plot_flag_patterns(df: pd.DataFrame, patterns: list, symbol: str):
    fig = create_candlestick_fig(df, symbol, title_suffix="with Flag Patterns")

    for pattern in patterns:
        pole_start = df.index[pattern['pole_start_idx']]
        pole_end = df.index[pattern['pole_end_idx']]
        flag_start = df.index[pattern['flag_start_idx']]
        flag_end = df.index[pattern['flag_end_idx']]
        breakout = df.index[pattern['breakout_idx']]
        
        ptype = pattern['type']
        color = 'green' if ptype == "Bullish Flag" else 'red'
        
        # 1. วาด Pole (เสาธง)
        fig.add_shape(
            type='line',
            x0=pole_start, y0=df['close'].loc[pole_start],
            x1=pole_end, y1=df['close'].loc[pole_end],
            line=dict(color=color, width=3, dash='solid')
        )
        
        # 2. วาด Flag (กรอบพักตัว)
        fig.add_vrect(
            x0=flag_start, x1=flag_end,
            fillcolor=color, opacity=0.15, layer="below", line_width=0
        )
        
        # 3. วาด Breakout Marker
        marker_symbol = 'triangle-up' if ptype == "Bullish Flag" else 'triangle-down'
        
        fig.add_trace(go.Scatter(
            x=[breakout], y=[df['close'].loc[breakout]],
            mode='markers',
            marker=dict(symbol=marker_symbol, size=12, color=color),
            name=f'{ptype} Breakout',
            showlegend=True
        ))
        
        # 4. กำกับข้อความ
        fig.add_annotation(
            x=flag_end, 
            y=df['high'].loc[flag_end] if ptype == "Bullish Flag" else df['low'].loc[flag_end],
            text=ptype,
            showarrow=False,
            font=dict(color=color, size=10, weight='bold'),
            yshift=10 if ptype == "Bullish Flag" else -10
        )
    
    fig.show()

In [3]:
# ==========================================================
# 5️⃣ Main Execution
# ==========================================================
if __name__ == '__main__':
    
    STOCK_SYMBOL = 'AOT'  # สัญลักษณ์หุ้นที่ต้องการ
    DAYS_LOOKBACK = 120   # ช่วงวันย้อนหลังที่ต้องการวิเคราะห์

    try:
        # 1. ดึงข้อมูล
        df_data = get_candlestick_data(STOCK_SYMBOL, DAYS_LOOKBACK)
        print(f"✅ ดึงข้อมูล {STOCK_SYMBOL} ได้ {len(df_data)} แถวสำหรับ {DAYS_LOOKBACK} วัน")
        
        
        # --- 🚩 1. ตรวจจับ Wyckoff Cycle ---
        print("\n--- 🔎 ตรวจจับ Wyckoff Cycle ---")
        wyckoff_events = detect_wyckoff_events(
            df_data, 
            price_distance=5, 
            volume_percentile=0.90
        )
        print(f"✅ พบ Wyckoff Cycle Phases: {len(wyckoff_events['WYCKOFF_CYCLE'])} ช่วง")
        
        # 2. วาดกราฟ Wyckoff Cycle
        plot_wyckoff_cycle(df_data, wyckoff_events, STOCK_SYMBOL)

        
        # --- 🚩 2. ตรวจจับ Flag Patterns ---
        print(f"\n--- 🔎 ตรวจจับ Flag Patterns ---")
        flag_patterns = detect_flag_patterns(
            df_data, 
            pole_length_ratio=0.08, # ลด ratio ลงเล็กน้อยเพื่อโอกาสพบ pattern มากขึ้น
            flag_duration_max=30
        )
        
        # 3. แสดงผลลัพธ์
        print(f"✅ พบ Flag Patterns ทั้งหมด: {len(flag_patterns)} รูปแบบ")
        if flag_patterns:
            print(f"  - Bullish Flags: {sum(1 for p in flag_patterns if p['type'] == 'Bullish Flag')} รูปแบบ")
            print(f"  - Bearish Flags: {sum(1 for p in flag_patterns if p['type'] == 'Bearish Flag')} รูปแบบ")
            
            # 4. วาดกราฟ Flag Patterns
            plot_flag_patterns(df_data, flag_patterns, STOCK_SYMBOL)
            
        
    except ValueError as e:
        print(f"\n❌ ข้อผิดพลาดข้อมูล: {e}")
    except Exception as e:
        print(f"\n❌ เกิดข้อผิดพลาดที่ไม่คาดคิด: {e}")

✅ ดึงข้อมูล AOT ได้ 80 แถวสำหรับ 120 วัน

--- 🔎 ตรวจจับ Wyckoff Cycle ---
✅ พบ Wyckoff Cycle Phases: 2 ช่วง


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.





--- 🔎 ตรวจจับ Flag Patterns ---
✅ พบ Flag Patterns ทั้งหมด: 0 รูปแบบ
